## Recognizing Handwritten Digits

For this goal, we'll use the MNIST (refer to http://yann.lecun.com/exdb/mnist/), a database of handwritten digits made up of a training set of 60,000 examples and a test set of 10,000 examples. Each MNIST image is in greyscale and it consists of 28x28 pixels.

Keras provides suitable libraries to load the dataset and split it into training sets and tests sets, used for assessing the performance. Data is converted to `float32` for supporting GPU computation and normalized to `[0, 1]`. In addition, we load the true labels `Y_train` and `Y_test` respectively and perform a one-hot encoding on them.

* The input layer has a neuron associated with each pixel in the image for a total of 28 x 28 = 784 neurons, one for each pixel in the MNIST images;
* Typically, the values associated with each pixel are normalized in the range [0, 1] (which means that the intensity of each pixel is divided by 255, the maximum intensity value);
* The final layer is a single neuron with activation function `softmax`, which is a generalization of the `sigmoid` function;

Once we defined the model, we have to compile it so that it can be executed by the Keras backend (either Theano or TensorFlow). There are a few choices to be made during compilation:

* We need to select the `optimizer` that is the algorithm used to update weights while we train our model;
* We need to select the `objective function` that is used by the optimizer to navigate the space of weights (frequently, objective functions are called `loss function`, and the process of optimization is defined as a process of loss minimization);
* We need to evaluate the trained model.

Some common choices for metrics (a complete list of Keras metrics is at https://keras.io/metrics/) are as follows:

* **Accuracy**: This is the proportion of correct predictions with respect to the targets;
* **Precision**: This denotes how many selected items are relevant for a multilabel classification;
* **Recal**: This denotes how many selected items are relevant for a multilabel classification.

Metrics are similar to objective functions, with the only difference that they are not used for training a model but only for evaluating a model.

Once the model is compiled, it can be then trained with the fit() function, which specifies a few parameters:

* **epochs**: This is the number of times the model is exposed to the training set. At each iteration, the optimizer tries to adjust the weights so that the objective function is minimized;
* **batch_size**: This is the number of training instances observed before the optimizer performs a weight update.

In [ ]:
from __future__ import print_function
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
np.random.seed(1671) # for reproducibility

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# network and training
NB_EPOCH = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 # number of outputs
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # how much training data is reserved for validation

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
RESHAPED = 784

# X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalize
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(Y_test, NB_CLASSES)

# Creates the model
model = Sequential()
model.add(Dense(NB_CLASSES, input_shape=(RESHAPED,)))
model.add(Activation('softmax'))
model.summary()

# Selects the optimizer and the evaluation metrics.
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

# Trains the model
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE,
                    epochs=NB_EPOCH,
                    verbose=VERBOSE,
                    validation_split=VALIDATION_SPLIT)

# Evaluates the model
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

**Insights**
* The network is trained on 48,000 samples, and 12,000 are reserved for validation;
* Once the neural model is built, it is then tested on 10,000 samples;
* we can notice that the program runs for 200 iterations, and each time, the accuracy improves;

This means that a bit less than one handwritten character out of ten is not correctly recognized. We can certainly do better than that.

### Improving our neural network

* A first improvement is to add additional layers to our network;
* So, after the input layer, we have a first dense layer with the `N_HIDDEN` neurons and an activation function `relu`;
* This layer is called _hidden_ because it is not directly connected to either the input of the output;
* After the first hidden layer, we have a second hidden layer, again with the `N_HIDDEN` neurons, followed by an output layer with 10 neurons.

In [ ]:
from __future__ import print_function
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
np.random.seed(1671) # for reproducibility

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Network and training
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 # number of outputs
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # how much training data is reserved for validation

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
RESHAPED = 784

# X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalize
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(Y_test, NB_CLASSES)

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,)))
model.add(Activation('relu'))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

# Selects the optimizer and the evaluation metrics.
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

# Trains the model
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE, epochs=NB_EPOCH,
                    verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# Evaluates the model
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

### Further improving our neural network

* The second improvement is to randomly drop with the dropout probability some of the values propagated inside our internal dense network of hidden layers;
* In Machine Learning, this is a well known form of regularization;
* It has been frequently observed that networks with random dropout in internal hidden layers can generalize better on unseen examples contained in test sets;
* One can think of this as each neuron becoming more capable because it knows it cannot depend on its neighbors;
* During testing, there is no dropout, so we are now using all our highly tuned neurons;
* It is generally a good approach to test how a net performs when some dropout function is adopted.

**OBS:** try first training the network with `NB_EPOCH` set to 20. Note that training accuracy should be above test accuracy, otherwise we're not training long enough. After testing it with 20, set the `NB_EPOCH` value to 250 and see the results.

In [ ]:
from __future__ import print_function
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
np.random.seed(1671) # for reproducibility

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Network and training
NB_EPOCH = 250
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 # number of outputs
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # how much training data is reserved for validation
DROPOUT = 0.3

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
RESHAPED = 784

# X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalize
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(Y_test, NB_CLASSES)

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,)))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

# Selects the optimizer and the evaluation metrics.
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

# Trains the model
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE, epochs=NB_EPOCH,
                    verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# Evaluates the model
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

### Testing different optimizers

* Let's focus on one popular training technique known as gradient descent (GD);
* The gradient descent can be seen as a hiker who aims at climbing down a mountain into a valley;
* Imagine a generic cost function `C(w)` in one single variable `w`;
* At each step `r`, the gradient is the direction of maximum increase;
* At each step, the hiker can decide what the leg length is before the next step, which is the `learning rate` in gradient descent jargon;
* If the learning rate is too small, the hiker will move slowly, but it's too high, the hiker will possibly miss the valley;
* In practice, we just choose the activation function, and Keras uses its backend (Tensorflow or Theano) for computing its derivative on our behalf;
* When we discuss backpropagation, we will discover that the minimization game is a bit more complex than our toy example;
* Keras implements a fast variant of gradient descent known as stochastic gradient descent (`SGD`) and two more advanced optimization techniques known as `RMSprop` and `Adam`.

In [ ]:
from __future__ import print_function
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils
np.random.seed(1671) # for reproducibility

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Network and training
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 # number of outputs
OPTIMIZER = Adam()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # how much training data is reserved for validation
DROPOUT = 0.3

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
RESHAPED = 784

# X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalize
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(Y_test, NB_CLASSES)

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,)))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

# Selects the optimizer and the evaluation metrics.
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

# Trains the model
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE, epochs=NB_EPOCH,
                    verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# Evaluates the model
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

* So far, we made progressive improvements; however, the gains are now more and more difficult;
* Note that we are optimizing with a dropout of 30%;
* For the sake of completeness, it could be useful to report the accuracy on the test only for other dropout values with `Adam` chosen as optimizer.

### Increasing the number of epochs

* We can make another attempt and increase the number of epochs used for training from 20 to 200;
* Unfortunately, this choice increases our computation time by 10, but it gives us no gain;
* **Learning is more about adopting smart techniques and not necessarily about the time spent in computations.**

### Controlling the optimizer learning rate

* There is another attempt we can make, which is changing the learning parameter for our optimizer;
* If you plot different values, you'll see that the optimal value is somewhere close to 0.001.

### Increasing the number of internal hidden neurons

* We can make yet another attempt, that is, changing the number of internal hidden neurons;
* We report the results of the experiments with an increasing number of hidden neurons;
* By increasing the complexity of the model, the run time increases significantly because there are more and more parameters to optimize.

### Increasing the size of batch computation

* Gradient descent tries to minimize the cost function on all the examples provided in the training sets;
* Stochastic gradient descent considers only `BATCH_SIZE`;
* If we check the behavior is by changing this parameter, we notice that the optimal accuracy value is reached for BATCH_SIZE=128.

### Adopting regularization for avoiding overfitting

* A model can become excessively complex in order to capture all the relations inherently expressed by the training data, which can bring two problems:
  - First, a complex model might require a significant amount of time to be executed;
  - Second, a complex model can achieve good performance on training data and not be able to generalize on unsee data.
* As a rule of thumb, if during the training we see that the loss increases on validation, after an initial decrease, then we have a problem of model complexity that overfits training;
* In order to solve the overfitting problem, we need a way to capture the complexity of a model, that is, how complex a model can be;
* A model is nothing more than a vector of weights. Therefore the complexity of a model can be conveniently represented as the number of nonzero weights;
* If we have two models, M1 and M2, achieving pretty much the same performance in terms of loss function, then we should choose the simplest model that has the minimum number of nonzero weights;
* Playing with regularization can be a good way to increase the performance of a network, in particular when there is an evident situation of overfitting.
* There 3 types of regularization in machine learning:
  - **L1 regularization** (also known as **lasso**): The complexity of the model is expressed as the sum of the absolute values of the weights;
  - **L2 regularization** (also known as **ridge**): The complexity of the model is expressed as the sum of the squares of the weights;
  - **Elastic net regularization**: The complexity of the model is captured by a combination of the two preceding techniques.

### Hyperparameter tuning

* For a given net, there are indeed multiple parameters that can be optimized (such as the number of hidden neurons, BATCH_SIZE, number of epochs, and many more);
* Hyperparameter tuning is the process of finding the optimal combination of those parameters that minimize cost functions.
* In other words, the parameters are divided into buckets, and different combinations of values are checked via a brute force approach.

### Predicting Output

* You can use the following method for predicting the output with Keras:
* `model.predict(X)`: This is used to predict the Y values;
* `model.evaluate()`: This is used to compute the loss values;
* `model.predict_classes()`: This is used to compute category outputs;
* `model.predict_proba()`: This is used to compute class probabilities.

## Getting Started with Keras

### What is a tensor?

* A tensor is nothing but a multidimensional array or matrix;
* Keras uses either Theano or TensorFlow to perform very efficient computations on tensors;
* Both the backends are capable of efficient symbolic computations on tensors, which are the fundamental building blocks for creating neural networks.

### Predefined Neural Network Layers

* **Regular dense**: A dense model is a fully connected neural network layer;
* **Recurrent neural networks -- simple LSTM and GRU**: Recurrent neural networks are a class of neural networks that exploit the sequential nature or their input. Such inputs could be a text, a speech, time series, and anything else where the occurrence of an element in the sequence is dependent on the elements that appeared before it;
* **Convolutional and pooling layers**: ConvNets are a class of neural networks using convolutional and pooling operations for progressively learning rather sophisticated models based on progressive levels of abstraction. It resembles vision models that have evolved over millions of years inside the human brain. People called it deep with 3-5 layers a few years ago, and now it has gone up to 100-200;
* **Regularization**: Regularization is a way to prevent overfitting. Multiple layers have parameters for regularization. One example is `Dropout`, but there are others;
* **Batch normalization**: It's a way to accelerate learning and generally achieve better accuracy;

### Losses functions

Losses functions (or objective functions, or optimization score function) can be classified into four categories:

* **Accuracy** which is used for classification problems;
* **Error loss**, which measures the difference between the values predicted and the values actually observed. There are multiple choices: `mse` (mean square error), `rmse` (root mean square error), `mae` (mean absolute error), `mape` (mean percentage error) and `msle` (mean squared logarithmic error);
* **Hinge loss**, which is generally used for training classifiers;
* **Class loss** is used to calculate the cross-entropy for classification problems (see https://en.wikipedia.org/wiki/Cross_entropy).

### Metrics

A metric function is similar to an objective function. The only difference is that the results from evaluating a metric are not used when training the model.

### Optimizers

Optimizers include `SGD`, `RMSprop`, and `Adam`.

## Deep learning with Convolutional Networks (ConvNets)

* Leverage spacial information and are suited for classifying images;
* Based on how our vision is based on multiple cortex levels, with each one recognizing more and more structured information;
* Two different types of layers, convolutional and pooling, are typically alternated.

### Local receptive fields

* To preserve spatial information, we represent each image with a matrix of pixels;
* A simple way to encode the local structure is to connect a submatrix of adjacent input neurons into one single hidden neuron (which is the **local receptive field**) belonging to the next layer;
* Of course, we can encode more information by having overlapping submatrices;
* In Keras, the size of each single submatrix is called _stride length_, and this is a hyperparameter that can be fine-tuned during the construction of our nets;
* Of course, we can have multiple feature maps that learn independently from each hidden layer.

![ConvNet example](ConvNet.gif)

* Rather than focus on one pixel at a time, ConvNets take in square patches of pixels and passes them through a _filter_ (or _kernel_), and the job of the filter is to find patterns in the pixels;
* We are going to take the dot product of the filter with this patch of the image channel. If the two matrices have high values in the same positions, the dot product’s output will be high. If they don’t, it will be low.
* We start in the upper lefthand corner of the image and we move the filter across the image step by step until it reaches the upper righthand corner. The size of the step is known as `stride`. You can move the filter to the right 1 column at a time, or you can choose to make larger steps;
* At each step, you take another dot product, and you place the results of that dot product in a third matrix known as an `activation map`;
* The width, or number of columns, of the activation map is equal to the number of steps the filter takes to traverse the underlying image;
* Since larger strides lead to fewer steps, a big stride will produce a smaller activation map.
* This is important, because the size of the matrices that convolutional networks process and produce at each layer is directly proportional to how computationally expensive they are and how much time they take to train.
* **A larger stride means less time and compute.**

### Max Pooling/Downsampling

![Max Pool example](MaxPool.png)

* The activation maps are fed into a downsampling layer, and like convolutions, this method is applied one patch at a time;
* In this case, max pooling simply takes the largest value from one patch of an image;
* Much information is lost in this step, which has spurred research into alternative methods. But downsampling has the advantage, precisely because information is lost, of decreasing the amount of storage and processing required;

### Average Pooling

* The alternative method to Max Pooling is simply taking the average of the regions, which is called _average pooling_.

### ConvNets Summary

![ConvNets Summary](ConvNetSummary.png)

In the image above you can see:

* The actual input image that is scanned for features;
* Activation maps stacked atop one another, one for each filter you employ;
* The activation maps condensed through downsampling;
* A new set of activation maps created by passing filters over the first downsampled stack;
* The second downsampling, which condenses the second set of activation maps;
* A fully connected layer that classifies output with one label per node.

There are various architectures of CNNs available which have been key in building algorithms which power and shall power AI as a whole in the foreseeable future:

1. LeNet
2. AlexNet
3. VGGNet
4. GoogLeNet
5. ResNet
6. ZFNet

### LeNet code in Keras



In [ ]:
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import mnist

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#define the ConvNet
class LeNet:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        # CONV => RELU => POOL
        # Here, 20 is the number of convolution kernels/filters to use, each one with the size 5x5
        # padding='same' means that padding is used
        # Output dimension is the same one of the input shape, so it will be 28 x 28
        # pool_size=(2, 2) represents the factors by which the image is vertically and horizontally downscaled
        model.add(Conv2D(20, kernel_size=5, padding="same", input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # CONV => RELU => POOL
        # A second convolutional stage with ReLU activations follows
        # In this case, we increase the number of convolutional filters learned to 50
        # Increasing the number of filters in deeper layers is a common technique used in deep learning
        model.add(Conv2D(50, kernel_size=5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # Flatten => RELU layers
        # Pretty standard flattening and a dense network of 500 neurons
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        # Softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

# Training parameters
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = Adam()
VALIDATION_SPLIT = 0.2
IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
NB_CLASSES = 10 # number of outputs
INPUT_SHAPE = (1, IMG_ROWS, IMG_COLS)

# data: shuffled and split between train and test sets
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
K.set_image_dim_ordering("th")

# consider them as float and normalize
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# we need a 60K x [1 x 28 x 28] shape as input to the CONVNET
X_train = X_train[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(Y_test, NB_CLASSES)

# initialize the optimizer and model
model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.summary()
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])
history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)

print("Test score:", score[0])
print('Test accuracy:', score[1])

# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()